# Geospatial Machine Learning Pipeline for Binary Classification

This notebook implements a PCA-based machine learning pipeline for geospatial binary classification.
Originally developed for archaeological site prediction but can be adapted for any geospatial 
binary classification problem (e.g., landslide susceptibility, urban planning, habitat modeling).

## Requirements:
1. Point data with coordinates and binary class labels
2. Raster data covering the study area
3. All data in same coordinate reference system

## Usage:
1. Update file paths in the configuration section
2. Modify class_type_mapping for your specific classes  
3. Adjust raster_files dictionary based on your available data
4. Run the pipeline

**Author:** [Your name]  
**Paper:** [Paper title/link when published]

## 1. Install Required Packages

In [ ]:
# Install required packages (uncomment if needed)
!pip install pandas rasterio numpy matplotlib seaborn scikit-learn xgboost scipy statsmodels imbalanced-learn

## 2. Import Libraries

In [ ]:
import pandas as pd
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import zscore
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve, auc
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings('ignore')

## 3. Configuration Section
**⚠️ UPDATE THESE PATHS AND SETTINGS FOR YOUR DATA ⚠️**

In [ ]:
# =============================================================================
# CONFIGURATION SECTION - UPDATE THESE PATHS AND SETTINGS FOR YOUR DATA
# =============================================================================

# TODO: Update these paths to your data directory
raster_dir = ''  # Directory containing your raster files
points_file = ''  # Excel file with point coordinates and classes

# Required data format:
# - Points file (Excel): Must contain columns 'POINT_X', 'POINT_Y', 'class'
# - Raster files: GeoTIFF format with same projection as points
# - All rasters should cover the same geographic extent

# Define your class/category mapping - customize based on your data
class_type_mapping = {
    '1': 'Class_1',    # Replace with your actual class names
    '2': 'Class_2',    # e.g., 'Urban', 'Forest', 'Positive', 'Negative', etc.
    '3': 'Class_3',
    '4': 'Class_4',
    '5': 'Class_5'
    # Add or remove classes as needed for your specific problem
}

# Define raster files - customize based on your available data
# Example:
raster_files = {
    'elevation': ['elevation.tif', 'elevation_1km.tif', 'elevation_5km.tif'],
    'slope': ['slope.tif', 'slope_1km.tif', 'slope_5km.tif'],
    'accessibility': ['walking_time.tif'],        # Optional: accessibility data
    'hydrology': ['nearest_river_order.tif'],     # Optional: hydrological data
    'land_cover': ['land_cover.tif'],             # Optional: land cover data
    'soil_texture': ['soil_texture.tif']          # Optional: surface/texture data
    # Add or remove variables based on your research needs
    # Examples: precipitation, temperature, distance_to_roads, etc.
}

# Model parameters (adjust as needed)
Z_THRESHOLD = 2.576  # For outlier removal (99% confidence)
MIN_PRECISION_THRESHOLD = 0.65  # Minimum acceptable precision
N_FOLDS = 5  # Number of cross-validation folds
RANDOM_STATE = 42

## 4. Helper Functions

In [ ]:
def extract_raster_values(raster_path, points):
    """Extract raster values at given point locations"""
    with rasterio.open(raster_path) as src:
        values = [next(src.sample([(x, y)]))[0] for x, y in zip(points['POINT_X'], points['POINT_Y'])]
    return np.array(values).flatten()

def read_raster(raster_path):
    """Read entire raster data for prediction mapping"""
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        profile = src.profile
    data = np.nan_to_num(data, nan=0)  # Replace NaNs with 0
    return data.flatten(), profile

def create_qq_plots(data, grid_size=(3, 5)):
    """Create Q-Q plots for data distribution analysis"""
    if isinstance(data, pd.DataFrame):
        n_plots = data.shape[1]
    else:
        n_plots = data.shape[1]

    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(20, 15))
    axes = axes.flatten()

    for i in range(min(n_plots, len(axes))):
        ax = axes[i]
        if isinstance(data, pd.DataFrame):
            stats.probplot(data.iloc[:, i].dropna(), dist="norm", plot=ax)
            ax.set_title(f'Q-Q Plot: {data.columns[i]}')
        else:
            stats.probplot(data[:, i], dist="norm", plot=ax)
            ax.set_title(f'Q-Q Plot: Component {i + 1}')
        ax.grid(True)

    # Remove empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

def print_pca_loadings_and_weights(pca, data):
    """Analyze and visualize PCA loadings"""
    loadings = pca.components_
    loadings_df = pd.DataFrame(loadings.T, index=data.columns, 
                              columns=[f'PC{i+1}' for i in range(loadings.shape[0])])

    print("Principal Component Loadings:")
    print(loadings_df.round(3))

    # Visualize loadings
    plt.figure(figsize=(12, 8))
    sns.heatmap(loadings_df, annot=True, cmap='coolwarm')
    plt.title('Principal Component Loadings')
    plt.show()

    # Calculate feature importance
    explained_variance_ratio = pca.explained_variance_ratio_
    feature_weights = loadings_df.copy()
    
    for i, ev in enumerate(explained_variance_ratio):
        feature_weights.iloc[:, i] *= np.sqrt(ev)
    
    overall_feature_weights = feature_weights.sum(axis=1)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(overall_feature_weights)), overall_feature_weights.values)
    plt.xlabel('Features')
    plt.ylabel('Overall Weight')
    plt.title('Overall Weights of Original Features after PCA')
    plt.xticks(range(len(overall_feature_weights)), overall_feature_weights.index, rotation=45)
    plt.tight_layout()
    plt.show()
    
    return loadings_df

## 5. Load and Prepare Data

In [ ]:
# Load point data
try:
    points_data = pd.read_excel(points_file)
    print(f"Loaded {len(points_data)} points")
    print(f"Columns: {list(points_data.columns)}")
    print(f"Class distribution:\n{points_data['class'].value_counts()}")
except FileNotFoundError:
    print(f"Error: Could not find points file at {points_file}")
    print("Please update the points_file path in the configuration section")

In [ ]:
# Extract raster values at point locations
# TODO: Update these based on your actual raster files
feature_data = {}
raster_data_dict = {} # To store flattened full raster data for prediction
main_profile = None # To store the profile of the first successfully loaded raster

try:
    first_raster_loaded = False
    for feature_category, files_in_category in raster_files.items():
        for file_name_in_config in files_in_category:
            full_raster_path = os.path.join(raster_dir, file_name_in_config)
            # Create a unique feature name, e.g., elevation_elevation or elevation_elevation_1km
            # You might want a more sophisticated naming scheme if file_name_in_config can be complex
            base_name = os.path.splitext(file_name_in_config)[0]
            current_feature_name = f"{feature_category}_{base_name}" # Or simplify if redundant

            if os.path.exists(full_raster_path):
                print(f"Processing: {current_feature_name} from {file_name_in_config}")
                # Extract values for points_data
                feature_data[current_feature_name] = extract_raster_values(
                    full_raster_path, points_data
                )
                # Read full raster for prediction map
                flat_raster_array, profile = read_raster(full_raster_path)
                raster_data_dict[current_feature_name] = flat_raster_array

                if not first_raster_loaded and profile:
                    main_profile = profile
                    first_raster_loaded = True
                elif first_raster_loaded and profile and \
                     (profile['height'] != main_profile['height'] or \
                      profile['width'] != main_profile['width']):
                    print(f"WARNING: Raster {file_name_in_config} has different dimensions than the first raster. This may cause issues.")
            else:
                print(f"WARNING: Raster file not found: {full_raster_path}")

    print(f"Extracted {len(feature_data)} features for points.")
    print(f"Loaded {len(raster_data_dict)} full raster layers for mapping.")

except Exception as e:
    print(f"Error loading or processing raster data: {e}")
    print("Please check your raster file paths and ensure files exist and are valid.")

## 6. Data Preprocessing and Transformation

In [ ]:
# Create feature DataFrame
if feature_data:
    # Raw data
    raw_data = pd.DataFrame(feature_data)
    
    print("Raw data statistics:")
    print(raw_data.describe())
    
    print("\nSkewness of original data:")
    print(raw_data.skew())
    
    # Create Q-Q plots for raw data
    create_qq_plots(raw_data)
else:
    print("No feature data loaded. Please check your file paths.")

In [ ]:
# Apply transformations to normalize data
# Customize these transformations based on your data distribution

if feature_data:
    # Example transformations - adjust based on your data
    transformed_data = raw_data.copy()
    
    # Apply log transformation to skewed variables
    for col in raw_data.columns:
        if raw_data[col].skew() > 1:  # If highly skewed
            transformed_data[col] = np.log1p(raw_data[col])  # log(1+x) transformation
    
    print("Skewness after transformation:")
    print(transformed_data.skew())
    
    # Create Q-Q plots for transformed data
    create_qq_plots(transformed_data)
    
    # Apply same transformations to full raster data
    raster_data = pd.DataFrame(raster_data_dict)
    for col in transformed_data.columns:
        if col in raster_data.columns and raw_data[col].skew() > 1:
            raster_data[col] = np.log1p(raster_data[col])

## 7. Outlier Removal

In [ ]:
# Remove outliers using z-score threshold
if 'transformed_data' in locals():
    # Calculate z-scores
    z_scores_data = np.abs(zscore(transformed_data))
    z_scores_raster = np.abs(zscore(raster_data))
    
    # Create masks for valid data
    mask_data = (z_scores_data < Z_THRESHOLD).all(axis=1)
    mask_raster = (z_scores_raster < Z_THRESHOLD).all(axis=1)
    
    # Apply masks
    data_cleaned = transformed_data[mask_data]
    raster_data_cleaned = raster_data[mask_raster]
    y_cleaned = points_data['class'][mask_data]
    
    print(f"Original data shape: {transformed_data.shape}")
    print(f"Data shape after cleaning: {data_cleaned.shape}")
    print(f"Removed {len(transformed_data) - len(data_cleaned)} outliers ({(len(transformed_data) - len(data_cleaned))/len(transformed_data)*100:.1f}%)")
else:
    print("No transformed data available for outlier removal")

## 8. Principal Component Analysis (PCA)

In [ ]:
# Perform PCA analysis
if 'data_cleaned' in locals():
    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data_cleaned)
    
    # Fit PCA
    pca = PCA()
    pca.fit(X_scaled)
    
    # Print explained variance
    print("Explained variance ratio:")
    for i, var in enumerate(pca.explained_variance_ratio_):
        print(f"PC{i+1}: {var:.3f} ({var*100:.1f}%)")
    
    print(f"\nCumulative explained variance:")
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    for i, var in enumerate(cumsum):
        print(f"PC1-{i+1}: {var:.3f} ({var*100:.1f}%)")
    
    # Analyze loadings
    loadings_df = print_pca_loadings_and_weights(pca, data_cleaned)
else:
    print("No cleaned data available for PCA")

In [ ]:
# Visualize PCA results
if 'pca' in locals():
    # Transform data to PC space
    X_pca = pca.transform(X_scaled)
    
    # Plot PC1 vs PC2
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_cleaned, alpha=0.7, cmap='viridis')
    plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
    plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
    plt.title("PCA: PC1 vs PC2 colored by class")
    plt.colorbar(scatter, label='Class')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Plot explained variance
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
    plt.xlabel('Principal Component')
    plt.ylabel('Explained Variance Ratio')
    plt.title('Explained Variance by Component')
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(cumsum) + 1), cumsum, 'bo-')
    plt.axhline(y=0.8, color='r', linestyle='--', label='80% variance')
    plt.axhline(y=0.9, color='g', linestyle='--', label='90% variance')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('Cumulative Explained Variance')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 9. Model Evaluation Functions

In [ ]:
def evaluate_model_with_kfold(X, y, model_class, pca_indices, model_params=None, k=5):
    """
    Evaluate any model using K-Fold cross-validation
    """
    if model_params is None:
        model_params = {}
    
    kf = KFold(n_splits=k, shuffle=True, random_state=RANDOM_STATE)
    precision_scores = []
    recall_scores = []
    f1_scores = []
    
    best_recall = -1
    best_model = None
    best_scaler = None
    best_pca = None
    
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        # Split data
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Scale data
        fold_scaler = StandardScaler()
        X_train_scaled = fold_scaler.fit_transform(X_train)
        X_test_scaled = fold_scaler.transform(X_test)
        
        # Apply PCA
        fold_pca = PCA(n_components=len(X.columns))
        X_train_pca = fold_pca.fit_transform(X_train_scaled)
        X_test_pca = fold_pca.transform(X_test_scaled)
        
        # Select components
        X_train_selected = X_train_pca[:, pca_indices]
        X_test_selected = X_test_pca[:, pca_indices]
        
        # Train model
        model = model_class(**model_params)
        model.fit(X_train_selected, y_train)
        
        # Get probabilities
        if hasattr(model, 'predict_proba'):
            y_prob = model.predict_proba(X_test_selected)[:, 1]
        else:
            # For models without predict_proba, use decision function
            y_prob = model.decision_function(X_test_selected)
        
        # Calculate precision-recall curve
        precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
        
        # Find optimal threshold
        optimal_idx = -1
        max_recall = -1
        
        for idx in range(len(thresholds)):
            if precision[idx] >= MIN_PRECISION_THRESHOLD:
                if recall[idx] > max_recall:
                    max_recall = recall[idx]
                    optimal_idx = idx
        
        if optimal_idx == -1:
            optimal_idx = np.argmax(precision[:-1])
        
        # Store best model
        if recall[optimal_idx] > best_recall:
            best_recall = recall[optimal_idx]
            best_model = model
            best_scaler = fold_scaler
            best_pca = fold_pca
        
        # Calculate F1 score
        f1 = 2 * (precision[optimal_idx] * recall[optimal_idx]) / (precision[optimal_idx] + recall[optimal_idx])
        
        precision_scores.append(precision[optimal_idx])
        recall_scores.append(recall[optimal_idx])
        f1_scores.append(f1)
        
        print(f"Fold {fold}: Precision={precision[optimal_idx]:.3f}, Recall={recall[optimal_idx]:.3f}, F1={f1:.3f}")
    
    print(f"\nAverage - Precision: {np.mean(precision_scores):.3f}±{np.std(precision_scores):.3f}")
    print(f"Average - Recall: {np.mean(recall_scores):.3f}±{np.std(recall_scores):.3f}")
    print(f"Average - F1: {np.mean(f1_scores):.3f}±{np.std(f1_scores):.3f}")
    
    return f1_scores, best_model, best_pca, best_scaler

def plot_probability_map(probability_map, profile, name):
    """Plot and save probability map"""
    # Create classification thresholds
    mean_prob = np.mean(probability_map)
    std_prob = np.std(probability_map)
    
    thresholds = [
        mean_prob - std_prob,
        mean_prob - 0.5 * std_prob,
        mean_prob,
        mean_prob + 0.5 * std_prob,
        mean_prob + std_prob
    ]
    
    # Classify probabilities
    classified_map = np.zeros_like(probability_map, dtype=np.uint8)
    for i, threshold in enumerate(thresholds):
        if i == 0:
            classified_map[probability_map < threshold] = 1
        else:
            classified_map[(probability_map >= thresholds[i-1]) & (probability_map < threshold)] = i + 1
    
    classified_map[probability_map >= thresholds[-1]] = len(thresholds) + 1
    
    # Plot
    plt.figure(figsize=(10, 8))
    cmap = ListedColormap(['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'])
    plt.imshow(classified_map, cmap=cmap)
    plt.title(f'Probability Map - {name}')
    plt.colorbar(label='Probability Class')
    plt.show()
    
    return classified_map

## 10. Model Comparison

In [ ]:
# Compare different models with different PC combinations
if 'data_cleaned' in locals() and 'y_cleaned' in locals():

    models_to_test = {
        'Random Forest': (RandomForestClassifier, {'n_estimators': 100, 'random_state': RANDOM_STATE}),
        'Logistic Regression': (LogisticRegression, {'random_state': RANDOM_STATE, 'max_iter': 1000, 'solver': 'liblinear'}),
        'K-Nearest Neighbors': (KNeighborsClassifier, {'n_neighbors': 5}),
        'SVM': (SVC, {'probability': True, 'random_state': RANDOM_STATE}),
        'XGBoost': (XGBClassifier, {'use_label_encoder': False, 'eval_metric': 'logloss', 'random_state': RANDOM_STATE})
    }

    # pc_combinations dictionary should be defined here as in your original notebook
    # It is used by the updated map generation cell.
    # Ensure this definition is robust, especially 'All PCs' if number of columns in data_cleaned changes.
    # For example, if data_cleaned is available:
    num_features = len(data_cleaned.columns) if 'data_cleaned' in locals() and hasattr(data_cleaned, 'columns') else 5 # Default to 5 if not available
    
    pc_combinations = {
        'All PCs': list(range(num_features)), # Dynamically set based on features
        'First 3 PCs': [i for i in [0, 1, 2] if i < num_features], # Ensure indices are valid
        'First 5 PCs': [i for i in [0, 1, 2, 3, 4] if i < num_features],
        # 'Top Variance PCs' might need to be defined after PCA analysis to know which PCs are top variance
        # For now, let's assume it's similar to 'First 5 PCs' or a user-defined list based on their PCA inspection.
        # Example: 'Top Variance PCs': [0, 1, 2, 3, 4] # User should adjust this based on their PCA results
    }
    # Ensure 'Top Variance PCs' or any other custom key used in evaluation is present in pc_combinations
    # If 'Top Variance PCs' was intended to be dynamic, that logic would need to be implemented after PCA.
    # For simplicity, if you had a specific set of indices for it:
    # pc_combinations['Top Variance PCs'] = [0, 1, 3] # Example, ensure indices are < num_features

    results = {}

    for model_name, (model_class, model_params) in models_to_test.items():
        print(f"\n{'='*50}")
        print(f"Testing {model_name}")
        print('='*50)

        model_results = {}

        for pc_name, pc_indices_list_for_eval in pc_combinations.items():
            # Ensure pc_indices_list_for_eval is not empty and all indices are valid for current data
            valid_pc_indices_for_eval = [idx for idx in pc_indices_list_for_eval if idx < num_features]
            if not valid_pc_indices_for_eval and pc_indices_list_for_eval: # If list was not empty but all indices were invalid
                 print(f"\n--- {pc_name} --- SKIPPING: PC indices {pc_indices_list_for_eval} are out of bounds for {num_features} features.")
                 model_results[pc_name] = None
                 continue
            if not valid_pc_indices_for_eval and not pc_indices_list_for_eval and pc_name != 'All PCs': # If list was empty initially (and not 'All PCs' which implies all)
                 print(f"\n--- {pc_name} --- SKIPPING: No PC indices defined.")
                 model_results[pc_name] = None
                 continue


            print(f"\n--- {pc_name} (using indices: {valid_pc_indices_for_eval}) ---")
            try:
                f1_scores, best_model, best_pca_fold, best_scaler_fold = evaluate_model_with_kfold( # Renamed for clarity
                    data_cleaned, y_cleaned, model_class, valid_pc_indices_for_eval, model_params, N_FOLDS
                )
                model_results[pc_name] = {
                    'f1_scores': f1_scores,
                    'mean_f1': np.mean(f1_scores),
                    'std_f1': np.std(f1_scores),
                    'best_model': best_model,
                    'best_pca': best_pca_fold, # Storing the PCA object from the best fold
                    'best_scaler': best_scaler_fold # Storing the Scaler object from the best fold
                }
            except Exception as e:
                print(f"Error with {model_name} - {pc_name}: {e}")
                model_results[pc_name] = None
                # import traceback # Optional: for more detailed error for user
                # traceback.print_exc()

        results[model_name] = model_results
else:
    print("No cleaned data available for model comparison")

## 11. Results Visualization

In [ ]:
# Create comparison plots
if 'results' in locals():
    # Prepare data for plotting
    plot_data = []
    
    for model_name, model_results in results.items():
        for pc_name, result in model_results.items():
            if result is not None:
                for f1_score in result['f1_scores']:
                    plot_data.append({
                        'Model': model_name,
                        'PC_Selection': pc_name,
                        'F1_Score': f1_score
                    })
    
    if plot_data:
        df_results = pd.DataFrame(plot_data)
        
        # Box plot comparison
        plt.figure(figsize=(15, 8))
        sns.boxplot(data=df_results, x='Model', y='F1_Score', hue='PC_Selection')
        plt.title('Model Performance Comparison Across Different PC Selections')
        plt.xticks(rotation=45)
        plt.legend(title='PC Selection', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        summary_stats = df_results.groupby(['Model', 'PC_Selection'])['F1_Score'].agg(['mean', 'std']).round(3)
        print("\nSummary Statistics:")
        print(summary_stats)

## 12. Generate Prediction Maps

In [ ]:
# Generate prediction maps for best performing models
if 'results' in locals() and 'raster_data' in locals() and 'pc_combinations' in locals(): # Added 'pc_combinations' check
    # Find best model overall
    best_overall_f1 = 0
    best_config = None
    # To store the actual pc_indices list used by the best model
    best_pc_indices_list = None

    for model_name, model_results in results.items():
        for pc_name, result in model_results.items():
            if result is not None and result['mean_f1'] > best_overall_f1:
                best_overall_f1 = result['mean_f1']
                best_config = (model_name, pc_name, result)
                # Store the pc_indices list that corresponds to this pc_name
                if pc_name in pc_combinations:
                    best_pc_indices_list = pc_combinations[pc_name]
                else:
                    # Fallback or error if pc_name isn't in pc_combinations,
                    # though it should be if results were generated based on it.
                    print(f"Warning: PC selection name '{pc_name}' not found in 'pc_combinations' dict. Using default for map.")
                    # Define a default or handle error appropriately
                    best_pc_indices_list = list(range(result['best_pca'].n_components_))


    if best_config and best_pc_indices_list is not None:
        model_name, pc_name, result = best_config
        print(f"Best model for map generation: {model_name} with {pc_name} (F1: {best_overall_f1:.3f})")
        print(f"Using PC indices for map: {best_pc_indices_list}")

        # Generate prediction map
        try:
            # Ensure raster_data has the same columns as the data used for training the scaler/pca
            # This alignment should ideally happen when raster_data (DataFrame) is first created/transformed.
            # Assuming 'data_cleaned.columns' represents the feature set the 'best_scaler' and 'best_pca' were trained on.
            if 'data_cleaned' in locals():
                 # Ensure column order and presence matches data_cleaned for the scaler
                raster_data_for_map = raster_data[data_cleaned.columns]
            else:
                print("Warning: 'data_cleaned' not found. Assuming 'raster_data' columns are already aligned for the scaler.")
                raster_data_for_map = raster_data


            # Scale raster data using the scaler from the best fold of the best model
            raster_scaled = result['best_scaler'].transform(raster_data_for_map)
            # Apply PCA transformation using the PCA object from the best fold
            raster_pca = result['best_pca'].transform(raster_scaled)

            # Select the appropriate principal components using the dynamically retrieved list
            # Ensure selected indices are within the bounds of available PCs from this specific PCA object
            valid_pc_indices_for_map = [idx for idx in best_pc_indices_list if idx < result['best_pca'].n_components_]
            if len(valid_pc_indices_for_map) != len(best_pc_indices_list):
                print(f"Warning: Some specified PC indices for mapping ({best_pc_indices_list}) were out of bounds for the {result['best_pca'].n_components_} PCs available from the best model's PCA. Using valid subset: {valid_pc_indices_for_map}")
            
            if not valid_pc_indices_for_map:
                print(f"Error: No valid PC indices to select for map generation with {pc_name}. Skipping map.")
            else:
                raster_selected = raster_pca[:, valid_pc_indices_for_map]

                # Predict probabilities
                if hasattr(result['best_model'], 'predict_proba'):
                    probabilities = result['best_model'].predict_proba(raster_selected)[:, 1]
                else: # For models like SVC without probability=True initially, or other decision_function based models
                    probabilities = result['best_model'].decision_function(raster_selected)
                    # Normalize to 0-1 range if it's not already a probability
                    if not (probabilities.min() >= 0 and probabilities.max() <= 1): # Basic check
                        probabilities = (probabilities - probabilities.min()) / (probabilities.max() - probabilities.min() + 1e-9) # add epsilon

                # Reshape to map
                if main_profile:
                    height, width = main_profile['height'], main_profile['width']
                    if probabilities.size == height * width:
                        probability_map = probabilities.reshape(height, width)

                        # Plot map
                        classified_map = plot_probability_map(probability_map, main_profile, f"{model_name}_{pc_name.replace(' ', '_')}")

                        print(f"Prediction map generated successfully!")
                        print(f"Map statistics - Mean: {np.mean(probability_map):.3f}, Std: {np.std(probability_map):.3f}")
                    else:
                        print(f"Error: Probability array size ({probabilities.size}) does not match map dimensions ({height*width}). Cannot reshape.")
                else:
                    print("Error: main_profile not available. Cannot determine map dimensions.")

        except Exception as e:
            print(f"Error generating prediction map for {model_name} with {pc_name}: {e}")
            import traceback
            traceback.print_exc() # Print full traceback for debugging
    elif not 'raster_data' in locals():
        print("Skipping map generation: 'raster_data' is not defined.")
    elif not 'pc_combinations' in locals():
        print("Skipping map generation: 'pc_combinations' is not defined (needed to get PC indices).")
    else:
        print("Skipping map generation: No best model configuration found or PC indices missing.")

## 13. Summary and Conclusions

In [ ]:
# Print summary of results
if 'results' in locals():
    print("="*60)
    print("SUMMARY OF RESULTS")
    print("="*60)
    
    for model_name, model_results in results.items():
        print(f"\n{model_name}:")
        for pc_name, result in model_results.items():
            if result is not None:
                print(f"  {pc_name}: F1 = {result['mean_f1']:.3f} ± {result['std_f1']:.3f}")
            else:
                print(f"  {pc_name}: Failed")
    
    print(f"\nBest Overall Performance:")
    if best_config:
        print(f"  Model: {best_config[0]}")
        print(f"  PC Selection: {best_config[1]}")
        print(f"  F1 Score: {best_config[2]['mean_f1']:.3f} ± {best_config[2]['std_f1']:.3f}")
    
    print("\n" + "="*60)
    print("Analysis complete! Check the prediction maps above.")
    print("="*60)
else:
    print("No results to summarize. Please check your data and configuration.")